## Protótipo PIDWise

In [2]:
# --- Imports ---
import numpy as np
import matplotlib.pyplot as plt
from collections import deque

try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output
except Exception as e:
    raise RuntimeError("Este notebook usa ipywidgets para os campos editáveis. Instale com: pip install ipywidgets") from e


# --- IMC para FOPTD -> PID (ISA paralelo dependente) ---
# Modelo de processo: Gp(s) = K * exp(-theta s) / (tau s + 1)
# Fórmulas IMC-PID (Rivera et al., forma comum para FOPTD):
#   Kp = (tau + 0.5*theta) / ( K * (lambda_ + 0.5*theta) )
#   Ti = tau + 0.5*theta
#   Td = (tau * theta) / (2*tau + theta)
def imc_pid_foptd(K, tau, theta, lambda_):
    if K <= 0 or tau <= 0 or lambda_ <= 0:
        raise ValueError("K, tau e lambda devem ser > 0.")
    Kp = (tau + 0.5*theta) / (K * (lambda_ + 0.5*theta))
    Ti = tau + 0.5*theta
    Td = (tau * theta) / (2.0*tau + theta) if (2.0*tau + theta) > 0 else 0.0
    return Kp, Ti, Td


# --- Simulação tempo discreto do FOPTD em malha fechada com PID (ISA paralelo dependente) ---
# Controlador: u = Kp*( e + (1/Ti) * ∫e dt + Td * de/dt )
# Processo (sem filtro): dx/dt = (-x + K*u_delay)/tau; y = x
# Atraso implementado com buffer de tamanho round(theta/dt)
def simulate_step_foptd_pid(K, tau, theta, Kp, Ti, Td, dt=0.05, t_final=None):
    # tempo de simulação padrão: 10 * max(tau, theta, 1)
    if t_final is None:
        t_final = 10.0 * max(tau, theta, 1.0)

    n_steps = int(np.ceil(t_final / dt)) + 1
    t = np.linspace(0.0, t_final, n_steps)

    # Sinal de referência: degrau unitário
    r = np.ones_like(t)

    # Estado do processo
    x = 0.0
    y = 0.0

    # Buffer para atraso de transporte
    n_delay = int(np.round(theta / dt))
    u_buffer = deque([0.0] * max(n_delay + 1, 1), maxlen=max(n_delay + 1, 1))

    # Termos do PID
    integ = 0.0
    e_prev = 0.0

    y_hist = np.zeros_like(t)
    u_hist = np.zeros_like(t)
    e_hist = np.zeros_like(t)

    for i, ti in enumerate(t):
        # erro
        e = r[i] - y

        # integral e derivada
        if Ti is not None and Ti > 0:
            integ += e * dt
            I_term = (integ / Ti)
        else:
            I_term = 0.0

        if Td is not None and Td > 0:
            dedt = (e - e_prev) / dt
            D_term = Td * dedt
        else:
            D_term = 0.0

        # controlador ISA paralelo dependente
        u = Kp * (e + I_term + D_term)

        # aplica atraso (pega u de theta segundos atrás)
        u_buffer.append(u)
        u_delayed = u_buffer[0]  # elemento mais antigo do buffer

        # dinâmica do processo (Euler explícito)
        dxdt = (-x + K * u_delayed) / tau
        x = x + dt * dxdt
        y = x

        # salva histórico
        y_hist[i] = y
        u_hist[i] = u
        e_hist[i] = e
        e_prev = e

    return t, y_hist, u_hist, e_hist

In [3]:
# --------- Widgets de entrada do processo ---------
K_w = widgets.FloatText(value=1.0, description='K (ganho):', step=0.1, layout=widgets.Layout(width='220px'))
tau_w = widgets.FloatText(value=10.0, description='tau (s):', step=0.5, layout=widgets.Layout(width='220px'))
theta_w = widgets.FloatText(value=2.0, description='theta (s):', step=0.5, layout=widgets.Layout(width='220px'))

# lambda padrão sugerido (seguro): max(0.1*tau, theta)
lambda_w = widgets.FloatText(value=max(0.1*tau_w.value, theta_w.value),
                             description='lambda:', step=0.5, layout=widgets.Layout(width='220px'))

# --------- Widgets dos ganhos do controlador (pré-preenchidos via IMC, mas editáveis) ---------
Kp_w = widgets.FloatText(value=0.0, description='Kp:', step=0.1, layout=widgets.Layout(width='200px'))
Ti_w = widgets.FloatText(value=0.0, description='Ti (s):', step=0.5, layout=widgets.Layout(width='200px'))
Td_w = widgets.FloatText(value=0.0, description='Td (s):', step=0.1, layout=widgets.Layout(width='200px'))

# Botões
btn_calc = widgets.Button(description='Calcular IMC', button_style='primary')
btn_sim  = widgets.Button(description='Simular degrau', button_style='success')

# Saída
out = widgets.Output()

def on_calc_clicked(_):
    with out:
        clear_output(wait=True)
        try:
            K = float(K_w.value)
            tau = float(tau_w.value)
            theta = float(theta_w.value)
            # lambda sugerido, mas usuário pode mudar
            lam = float(lambda_w.value)
            Kp, Ti, Td = imc_pid_foptd(K, tau, theta, lam)
            Kp_w.value = float(Kp)
            Ti_w.value = float(Ti)
            Td_w.value = float(Td)
            print("Ganhos IMC atribuídos aos campos (você pode editar se quiser).")
        except Exception as e:
            print("Erro no cálculo IMC:", e)

def on_sim_clicked(_):
    with out:
        clear_output(wait=True)
        try:
            # lê processo e controlador
            K = float(K_w.value)
            tau = float(tau_w.value)
            theta = float(theta_w.value)
            Kp = float(Kp_w.value)
            Ti = float(Ti_w.value)
            Td = float(Td_w.value)

            # passo e tempo final simples (automático)
            dt = 0.05
            t_final = 10.0 * max(tau, theta, 1.0)

            t, y, u, e = simulate_step_foptd_pid(K, tau, theta, Kp, Ti, Td, dt=dt, t_final=t_final)

            # Plot da resposta ao degrau unitário (sem filtro)
            plt.figure(figsize=(7.0, 4.0))
            plt.plot(t, y, label='PV (saída)')
            plt.plot(t, np.ones_like(t), '--', label='SP (1.0)')
            plt.xlabel('Tempo (s)')
            plt.ylabel('Valor')
            plt.title('Resposta ao degrau unitário (FOPTD + PID ISA paralelo)')
            plt.legend()
            plt.grid(True)
            plt.show()

            # (opcional) Mostrar também o sinal de controle U — útil para checar esforço
            plt.figure(figsize=(7.0, 3.5))
            plt.plot(t, u, label='u (sinal de controle)')
            plt.xlabel('Tempo (s)')
            plt.ylabel('u')
            plt.title('Ação de controle')
            plt.grid(True)
            plt.show()

            print(f"Parâmetros usados: K={K:.4g}, tau={tau:.4g}s, theta={theta:.4g}s | Kp={Kp:.4g}, Ti={Ti:.4g}s, Td={Td:.4g}s, lambda={float(lambda_w.value):.4g}")

        except Exception as e:
            print("Erro na simulação:", e)

btn_calc.on_click(on_calc_clicked)
btn_sim.on_click(on_sim_clicked)

# Layout
row1 = widgets.HBox([K_w, tau_w, theta_w, lambda_w, btn_calc])
row2 = widgets.HBox([Kp_w, Ti_w, Td_w, btn_sim])

display(row1, row2, out)

# Inicializa com cálculo IMC automático para já preencher Kp, Ti, Td
on_calc_clicked(None)

Output()